In [12]:
# Open the file
with open('./text/gatsby.txt') as file:
    raw_text = file.read()

In [2]:
# Preprocess that bitch
worked_text = raw_text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('-','').replace("'", "").replace('”', '').replace('"', '').replace('`', '').replace('´', '').replace('‘', '').replace('’', '').replace(',', '').replace(':', '').replace(';', '').replace('  ', ' ').replace('      ', ' ').replace('\u200a', ' ')

In [3]:
# Create vocabulary

import torch

vocab_letters = set(worked_text)
vocab = {}
for i, letter in enumerate(vocab_letters):
    value_matrix = torch.zeros(len(vocab_letters))
    value_matrix[i] = 1
    vocab[letter] = value_matrix
print(vocab)

{'D': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]), 'H': tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]), 'C': tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 

In [4]:
word_length = 15
X_set = torch.zeros(len(worked_text) - word_length, word_length, len(vocab))
y_set = torch.zeros(len(worked_text) - word_length)

for i in range(len(worked_text) - word_length):
    x = torch.zeros(word_length, len(vocab))
    for j in range(word_length):
        x[j] = vocab[worked_text[i+j]]
    X_set[i] = x
    y_set[i] = torch.argmax(vocab[worked_text[i+word_length]])


In [5]:
from torch.utils.data import Dataset

class Gatsbyset(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, i):
        return (self.X[i], self.y[i])
    
    def __len__(self):
        return self.X.shape[0]

In [6]:
print(len(y_set))
dataset = Gatsbyset(X_set, y_set)
train_len = int(len(y_set)*0.95)
train_set, val_set = torch.utils.data.random_split(dataset, [train_len, len(y_set) - train_len])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=8, shuffle=True, num_workers=0)

721703


In [7]:
class GatsbyLSTM(torch.nn.Module):
    
    def __init__(self, feature_size, hidden_size, num_stacks, out_size):
        super().__init__()
        self.LSTM = torch.nn.LSTM(feature_size, hidden_size, num_stacks, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, out_size)
        
    def forward(self, x):
        out, (h, c) = self.LSTM(x)
        return self.linear(h)[0]
    

In [8]:
import datetime
from tqdm import tqdm

def train(model, optimizer, loss_func, num_epochs, dataloader):
    model.train()
    for epoch in range(num_epochs):
        loss_train = 0.0
        train_bar = tqdm(total=len(dataloader), desc="Epoch")
        
        for xb, yb in dataloader:
            yb = yb.long()
            out = model(xb)
            loss = loss_func(out, yb)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_train += loss.item()
            train_bar.update()
        
        print(str(datetime.datetime.now()) + ' | Epoch: ' + str(epoch) + ' | Training loss: ' + str(loss_train/len(dataloader)))
        train_bar.close()

In [9]:
model = GatsbyLSTM(len(vocab), len(vocab), 1, len(vocab))
opt = torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()
train(model, opt, loss, 60, train_loader)

Epoch:   0%|          | 8/85703 [00:00<18:12, 78.46it/s]

2021-04-30 18:25:39.440015 | Epoch: 0 | Training loss: 1.688761531547208


Epoch:   0%|          | 8/85703 [00:00<17:59, 79.41it/s]

2021-04-30 18:33:33.841706 | Epoch: 1 | Training loss: 1.3795943035333178


Epoch:   0%|          | 31/85703 [00:00<08:53, 160.64it/s]

...

2021-05-01 13:18:48.768977 | Epoch: 82 | Training loss: 1.1607389851280243


Epoch:   0%|          | 11/85703 [00:00<13:14, 107.85it/s]

2021-05-01 13:26:51.778760 | Epoch: 83 | Training loss: 1.161004090716207


Epoch: 100%|██████████| 85703/85703 [08:11<00:00, 174.30it/s]


2021-05-01 13:35:03.469383 | Epoch: 84 | Training loss: 1.1602443698786042


In [10]:
# Initiate sentence to build off
seq_seed = "in my yo"
seq_seed_tensor = torch.zeros((1, word_length, len(vocab)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab[c]

# Create reverse-dictionary to translate vectors to letters
reversed_vocab = {}
for key in vocab:
    reversed_vocab[torch.argmax(vocab[key]).item()] = key

def generate_text(model, seed):
    for _ in range(500):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0, :-1] = seed[0, :-1].clone()
        seed[0, :-1] = vocab[reversed_vocab[torch.argmax(next_c).item()]]
        #print(reversed_vocab[torch.argmax(next_c).item()], end='')
        
#Generating text without random sampling repeats letter infinitely
generate_text(model, seq_seed_tensor)


In [11]:
import numpy as np

seq_seed = "in my y"
seq_seed_tensor = torch.zeros((1, word_length, len(vocab)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab[c]


def generate_text_sampling(model, seed):
    for _ in range(500):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0, :-1] = seed[0, :-1].clone()
        test_c = np.random.choice(len(vocab), 1, p=next_c.cpu().detach().numpy())
        seed[0, :-1] = vocab[reversed_vocab[test_c[0]]]
        print(reversed_vocab[test_c[0]], end='')

# 
generate_text_sampling(model, seq_seed_tensor)

—n mnkl—l.gyl oe l mb!l.vn—l rcyeRoe r mr?vn—e oytn K.—nl rvn—l l?il oe—l?il—oe?rd i?il mttn _—oes )en—u_ r.v e il..v—gil rf mtn—u_ioen—l  mnkdtnkvn—l_ rsrkdbl—w AGybl rr.vnkl.r—l o.vre rftnn—e—l lu_NSe e_!l.vnkl NR mtn .l odhl l—_b.v  i.vncd!h!n—db!h mtn—l l—l rf!f!il rtn—l?tnjvrs e re!l—l iemr—l   il—xe Mu_..vn—l_w rtnkmtil  h rfbl—)oe.gesivnkdtn—snl—hen— e Sel re il mtnl mtn—Bkm?ah rsm.vn)oe!lNAGoe!il.dges—oil)es.vn_rkl NILe.ve—u_v ml.v A_fil l?w!l lyoyr?.vil rfs!il r_. re.il.v l—syyatnkl.vrc